In [5]:
import os
import pandas as pd
import json

def parse_data(filename):
    df1 = pd.read_csv(filename)
    df2 = df1.rename(columns={"trnOpDate":"乘車日",
            "staCode":"車站代碼",
            "gateInComingCnt":"進站人數",
            "gateOutGoingCnt":"出站人數"})
    with open('車站基本資料集.json',encoding='utf-8') as file:
        json_data = json.load(file)
        stations_info = pd.DataFrame(json_data,columns=['stationCode','stationName'])
        stations_info = stations_info.astype({'stationCode':'int32'})
    df3 = pd.merge(df2,stations_info,left_on='車站代碼',right_on='stationCode')
    df4 = df3.rename(columns={'stationName':'車站名稱'})
    df5 = df4.reindex(columns=['乘車日','車站名稱','進站人數','出站人數'])
    df5['乘車日'] = pd.to_datetime(df5['乘車日'].astype(str))
    return df5

def main():
    current_dir = os.getcwd()
    csv_dir = os.path.join(current_dir,'每日各站進出站人數')
    files_and_dirs = os.listdir(csv_dir)
    all_csv_files = [filename for filename in files_and_dirs if '每日各站進出站人數' in filename]
    sorted_csv_files = sorted(all_csv_files)
    all_final_dfs = [] 
    for filename in sorted_csv_files:
        abs_filename = os.path.join(csv_dir,filename)
        all_final_dfs.append(parse_data(abs_filename))
    df_done = pd.concat(all_final_dfs)
    df_done1 = df_done.set_index('乘車日')
    return df_done1 


if __name__ == '__main__':
    df_done1 = main()

    

In [6]:
df_done2 = df_done1.query('車站名稱 in ("臺北","臺中","高雄")')

In [7]:
df_done2

,車站名稱,進站人數,出站人數
乘車日,,,
2019-04-23,臺北,55931,53907
2019-04-23,臺中,18885,18430
2019-04-23,高雄,13281,12606
2019-04-24,臺北,55970,54862
2019-04-24,臺中,18773,18509
...,...,...,...
2023-12-30,臺中,42661,43502
2023-12-30,高雄,18901,19072
2023-12-31,臺北,62141,90865


In [8]:
years = df_done2.index.to_series().dt.year.values
months = df_done2.index.to_series().dt.month.values


In [14]:
groupby = df_done2.groupby(by=[years, months,'車站名稱'])
df2 = groupby[['進站人數','出站人數']].sum()
df2

進站人數     出站人數
        車站名稱                  
2019 4  臺中     196819   196322
        臺北     499349   484128
        高雄     131501   129092
     5  臺中     756624   753259
        臺北    1881659  1844757
...               ...      ...
2023 11 臺北    1917620  1878372
        高雄     432731   434283
     12 臺中     861473   854234
        臺北    2033892  2015138
        高雄     480623   485788

[171 rows x 2 columns]

In [15]:
df2.unstack(level='車站名稱')

進站人數                     出站人數                 
車站名稱         臺中       臺北      高雄      臺中       臺北      高雄
2019 4   196819   499349  131501  196322   484128  129092
     5   756624  1881659  496946  753259  1844757  501062
     6   806261  1909510  516769  811109  1883199  515675
     7   813097  2036875  544617  741690  2033139  548130
     8   826856  2057971  528537  770365  2047495  528947
     9   811352  1861350  513354  816947  1861246  510600
     10  821320  2021092  532588  821740  2007138  534422
     11  798980  1957780  511155  799602  1935330  512241
     12  865393  1999044  538406  870876  2012196  542660
2020 1   853257  1894584  532721  851720  1851622  532198
     2   662240  1500211  391059  674267  1476076  388085
     3   566732  1362520  349488  563303  1351762  350412
     4   473254  1163616  282979  474284  1144422  281759
     5   582403  1383665  332425  583891  1359246  332177
     6   667794  1577529  403193  672598  1553532  404317
     7   745164  1785073  455539  755891  1748522  454959
     8   780932  1836197  477900  791989  1826697  475392
     9   680812  1694920  420352  683041  1660683  421117
     10  777955  1800929  465307  789753  1776930  468930
     11  725054  1714995  433081  734713  1705064  433672
     12  759804  1734442  458309  765387  1709115  461877
2021 1   679954  1541818  417812  684093  1530374  417228
     2   699411  1439767  420671  709872  1418680  424129
     3   689972  1661306  423728  695948  1651843  427297
     4   707678  1648519  425802  709312  1615052  427424
     5   400709   941069  253633  403865   931124  254419
     6   100667   352983   78896  101790   354242   78602
     7   156347   486983  110182  158890   478779  109769
     8   335429   897180  204229  340135   879687  206019
     9   444338  1064347  266194  444401  1049596  268339
     10  582067  1361807  340491  583004  1336819  344892
     11  590041  1469829  352573  590804  1447943  357140
     12  681566  1622531  393457  680485  1617439  403794
2022 1   581566  1369769  324785  578734  1299137  323423
     2   548708  1173413  300946  559293  1218193  306106
     3   589879  1537641  343998  586534  1512988  348242
     4   524714  1255721  302446  525196  1223943  303592
     5   345864   873444  213052  342128   877792  214884
     6   380827  1078938  218836  382892  1061113  222175
     7   569848  1437376  317279  573803  1414907  320510
     8   637010  1558129  367222  641151  1546615  368141
     9   604952  1465511  339369  606791  1434527  343951
     10  658218  1528067  366792  663179  1508896  370025
     11  628440  1577913  351005  628654  1558609  354034
     12  716617  1709109  384875  713199  1697585  393045
2023 1   780852  1631817  411788  780312  1595645  412376
     2   703190  1642325  380023  705011  1615980  380976
     3   676481  1742523  385126  667278  1696197  392025
     4   760188  1742602  421624  764348  1711153  420199
     5   707246  1727951  385261  702431  1713472  386588
     6   711998  1709392  386020  707219  1675589  388091
     7   763165  1856836  400562  764193  1830622  397492
     8   778790  1891981  422659  779324  1866289  421531
     9   742373  1850323  394908  729035  1788890  394728
     10  810733  1948006  448739  811878  1936032  450459
     11  748596  1917620  432731  735726  1878372  434283
     12  861473  2033892  480623  854234  2015138  485788